In [ ]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pycountry
import re
from collections import defaultdict
from scipy.stats import mstats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# crop_and_livestock feature selection and ISO3

df_crop = pd.read_csv("crop_and_livestock.csv")

df_crop = df_crop[['Area', 'Year', 'Item', 'Element', 'Value']]

FALLBACK_ISO3 = {
    "Iran (Islamic Republic of)": "IRN",
    "United Kingdom of Great Britain and Northern Ireland": "GBR"
}

def clean_country_name(name):
    name = name.strip()
    name = re.sub(r"\s+", " ", name)
    return name

def country_to_iso3(name):
    name = clean_country_name(name)
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return FALLBACK_ISO3.get(name, None)

df_crop['ISO3'] = df_crop['Area'].apply(country_to_iso3)
df_crop_new = df_crop.drop('Area', axis = 1)

important_items = [
    "Hen eggs in shell, fresh",
    "Onions and shallots, dry (excluding dehydrated)",
    "Meat of chickens, fresh or chilled",
    "Potatoes",
    "Other vegetables, fresh n.e.c.",
    "Meat of cattle with the bone, fresh or chilled",
    "Tomatoes",
    "Wheat",
    "Meat of sheep, fresh or chilled",
    "Barley"
]

df_crop_filtered = df_crop_new[
    df_crop_new['Item'].isin(important_items)
]

df_crop_pivot = df_crop_filtered.pivot_table(
    index=["ISO3", "Year"],
    columns=["Item","Element"],
    values="Value",
    aggfunc="mean"
).reset_index()

df_crop_pivot.columns = [
    "_".join([str(c) for c in col if c]).replace(" ", "_")
    if isinstance(col, tuple) else str(col)
    for col in df_crop_pivot.columns
]

#print(df_crop_pivot)
print("Shape:", df_crop_pivot.shape)

Shape: (600, 33)


In [ ]:
#DISASTER + CROP_AND_LIVESTOCK

df_disaster = pd.read_csv("disasters.csv")

df_disaster = df_disaster[
    ["ISO", "Start Year", "Disaster Type", "Disaster Subtype",
     "Total Damage, Adjusted ('000 US$)",
     "No. Affected", "Total Deaths"]
]

df_disaster.rename(columns={"Start Year": "Year"}, inplace=True)
df_disaster.rename(columns={"ISO": "ISO3"}, inplace=True)

df_disaster_agg = df_disaster.groupby(["ISO3", "Year"], as_index=False).agg({
    "Total Damage, Adjusted ('000 US$)": "sum",
    "No. Affected": "sum",
    "Total Deaths": "sum"
})

crop_disaster_merged = pd.merge(df_crop_pivot, df_disaster_agg, on=["ISO3", "Year"], how="left")

#print(df_merged)

print("Columns in merged dataset:", crop_disaster_merged.columns.tolist())
print("Shape:", crop_disaster_merged.shape)

Columns in merged dataset: ['ISO3', 'Year', 'Barley_Area_harvested', 'Barley_Production', 'Barley_Yield', 'Hen_eggs_in_shell,_fresh_Laying', 'Hen_eggs_in_shell,_fresh_Production', 'Hen_eggs_in_shell,_fresh_Yield', 'Hen_eggs_in_shell,_fresh_Yield/Carcass_Weight', 'Meat_of_cattle_with_the_bone,_fresh_or_chilled_Producing_Animals/Slaughtered', 'Meat_of_cattle_with_the_bone,_fresh_or_chilled_Production', 'Meat_of_cattle_with_the_bone,_fresh_or_chilled_Yield/Carcass_Weight', 'Meat_of_chickens,_fresh_or_chilled_Producing_Animals/Slaughtered', 'Meat_of_chickens,_fresh_or_chilled_Production', 'Meat_of_chickens,_fresh_or_chilled_Yield/Carcass_Weight', 'Meat_of_sheep,_fresh_or_chilled_Producing_Animals/Slaughtered', 'Meat_of_sheep,_fresh_or_chilled_Production', 'Meat_of_sheep,_fresh_or_chilled_Yield/Carcass_Weight', 'Onions_and_shallots,_dry_(excluding_dehydrated)_Area_harvested', 'Onions_and_shallots,_dry_(excluding_dehydrated)_Production', 'Onions_and_shallots,_dry_(excluding_dehydrated)_Yield

In [ ]:
#CORE + EMP_UNEMP

df_core = pd.read_csv("Core_economic_indicators.csv")
df_emp  = pd.read_csv("Employment_Unemployment.csv")

def reshape_dataset(df, value_col_name, selected_series):
    df['ISO3'] = df['Country Code']
    year_cols = [col for col in df.columns if '[YR' in str(col)]

    df_long = df.melt(
        id_vars=['Series Name', 'ISO3'],
        value_vars=year_cols,
        var_name='Year_Column',
        value_name=value_col_name
    )
    df_long['Year'] = df_long['Year_Column'].str.extract(r'(\d{4})').astype(int)
    df_long.drop('Year_Column', axis=1, inplace=True)
    df_long.replace('..', np.nan, inplace=True)
    df_long.dropna(subset=[value_col_name], inplace=True)
    df_long[value_col_name] = pd.to_numeric(df_long[value_col_name], errors='coerce')

    df_long = df_long[df_long['Series Name'].isin(selected_series)]
    return df_long

core_selected = [
    'Imports of goods and services (% of GDP)',
    'Exports of goods and services (% of GDP)',
    'Trade (% of GDP)',
    'Inflation, consumer prices (annual %)',
    'GDP growth (annual %)',
    'GDP per capita (current US$)',
    'GDP (current US$)'
]


emp_selected = [
    'Employment to population ratio, 15+, female (%) (modeled ILO estimate)',
    'Employment to population ratio, ages 15-24, male (%) (national estimate)',
    'Employment to population ratio, ages 15-24, female (%) (national estimate)',
    'Unemployment with basic education, male (% of male labor force with basic education)',
    'Unemployment with basic education, female (% of female labor force with basic education)'
]

df_core_long = reshape_dataset(df_core, "Economic_Value", core_selected)
df_emp_long  = reshape_dataset(df_emp, "Employment_Value", emp_selected)

df_core_wide = df_core_long.pivot_table(
    index=['ISO3', 'Year'],
    columns='Series Name',
    values='Economic_Value'
).reset_index()

df_emp_wide = df_emp_long.pivot_table(
    index=['ISO3', 'Year'],
    columns='Series Name',
    values='Employment_Value'
).reset_index()

df_core_emp = pd.merge(df_core_wide, df_emp_wide, on=['ISO3', 'Year'], how='inner')

print("Columns in merged dataset:", df_core_emp.columns.tolist())
print("Shape:", df_core_emp.shape)
#print(df_core_emp)

Columns in merged dataset: ['ISO3', 'Year', 'Exports of goods and services (% of GDP)', 'GDP (current US$)', 'GDP growth (annual %)', 'GDP per capita (current US$)', 'Imports of goods and services (% of GDP)', 'Inflation, consumer prices (annual %)', 'Trade (% of GDP)', 'Employment to population ratio, 15+, female (%) (modeled ILO estimate)', 'Employment to population ratio, ages 15-24, female (%) (national estimate)', 'Employment to population ratio, ages 15-24, male (%) (national estimate)', 'Unemployment with basic education, female (% of female labor force with basic education)', 'Unemployment with basic education, male (% of male labor force with basic education)']
Shape: (625, 14)


In [ ]:
# First 4

df_core_emp['Year'] = df_core_emp['Year'].astype(int)
crop_disaster_merged['Year'] = crop_disaster_merged['Year'].astype(int)

merged_all = pd.merge(
    crop_disaster_merged,
    df_core_emp,
    on=["ISO3", "Year"],
    how="left",
    suffixes=('_crop_disaster', '_core_emp')
)

print("Shape:", merged_all.shape)

Shape: (600, 48)


In [ ]:
# Feature selection and ISO3 in Population

df_pop = pd.read_csv("population_and_demographics.csv")

FALLBACK_ISO3 = {
    "Iran (Islamic Republic of)": "IRN",
    "United Kingdom of Great Britain and Northern Ireland": "GBR",
    "United States of America": "USA"
}

def clean_country_name(name):
    name = name.strip()
    name = re.sub(r"\s+", " ", name)
    return name

def country_to_iso3(name):
    name = clean_country_name(name)
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return FALLBACK_ISO3.get(name, None)

df_pop["ISO3"] = df_pop["Area"].apply(country_to_iso3)

unmapped = df_pop[df_pop["ISO3"].isnull()]["Area"].unique()
if len(unmapped) > 0:
    print("Unmapped country names:", unmapped)
else:
    print("All country names converted successfully!")

df_pop = df_pop[["ISO3", "Element", "Year", "Value"]].copy()

df_pop_pivot = df_pop.pivot_table(
    index=["ISO3", "Year"],
    columns="Element",
    values="Value",
    aggfunc="mean"
).reset_index()


df_pop_pivot.columns = [
    re.sub(r'_+', '_', str(col).replace(" ", "_").replace("/", "_per_").replace("-", ""))
    for col in df_pop_pivot.columns
]

#print(df_pop_pivot)
print("Shape:", df_pop_pivot.shape)

All country names converted successfully!
Shape: (600, 7)


In [ ]:
#MERGE POPULATION AND RESILIENCE

df_resilience = pd.read_csv("Resiliance.csv")

df_resilience['ISO3'] = df_resilience['Country Code']

year_columns = [col for col in df_resilience.columns if '[YR' in str(col)]

df_resilience_long = pd.melt(
    df_resilience,
    id_vars=['Country Name', 'Series Name', 'ISO3'],
    value_vars=year_columns,
    var_name='Year_Column',
    value_name='Resilience_Value'
)

df_resilience_long['Year'] = df_resilience_long['Year_Column'].str.extract(r'(\d{4})').astype(int)
df_resilience_long = df_resilience_long.drop('Year_Column', axis=1)

df_resilience_long = df_resilience_long.replace('..', np.nan)
df_resilience_long = df_resilience_long.dropna(subset=['Resilience_Value', 'ISO3', 'Year'])
df_resilience_long['Resilience_Value'] = pd.to_numeric(df_resilience_long['Resilience_Value'], errors='coerce')
df_resilience_long = df_resilience_long.dropna(subset=['Resilience_Value'])

df_resilience_long['Year'] = df_resilience_long['Year'].astype(int)
df_pop['Year'] = df_pop['Year'].astype(int)

df_resilience_wide = df_resilience_long.pivot_table(
    index=['ISO3', 'Year'],
    columns='Series Name',
    values='Resilience_Value',
    aggfunc='mean'
).reset_index()

df_resilience_wide.columns.name = None
df_resilience_wide.columns = [str(col).replace(" ", "_").replace("/", "_per_") for col in df_resilience_wide.columns]

df_merged_pop_res = pd.merge(df_pop_pivot, df_resilience_wide, on=['ISO3', 'Year'], how='left')

#print(df_merged_pop_res)
print("Shape:", df_merged_pop_res.shape)
print(df_merged_pop_res)


Shape: (600, 10)
    ISO3  Year  Rural_population  Total_Population_Both_sexes  \
0    AFG  2000         15657.474                    20130.327   
1    AFG  2001         16318.324                    20284.307   
2    AFG  2002         17086.910                    21378.117   
3    AFG  2003         17909.063                    22733.049   
4    AFG  2004         18692.107                    23560.654   
..   ...   ...               ...                          ...   
595  USA  2019         57727.196                   337790.067   
596  USA  2020         57456.395                   339436.159   
597  USA  2021         57168.078                   340161.441   
598  USA  2022         56862.780                   341534.045   
599  USA  2023         56539.549                   343477.335   

     Total_Population_Female  Total_Population_Male  Urban_population  \
0                  10035.682              10094.646          4436.282   
1                  10104.514              10179.792     

In [ ]:
#Merge Social Population Resilience

df_social = pd.read_csv("Social_and_welfare.csv")

df_social['ISO3'] = df_social['Country Code']

year_columns_social = [col for col in df_social.columns if '[YR' in str(col)]

df_social_long = pd.melt(
    df_social,
    id_vars=['Series Name', 'ISO3'],
    value_vars=year_columns_social,
    var_name='Year_Column',
    value_name='Social_Value'
)

df_social_long['Year'] = df_social_long['Year_Column'].str.extract(r'(\d{4})').astype(int)
df_social_long.drop('Year_Column', axis=1, inplace=True)

df_social_long.replace('..', np.nan, inplace=True)
df_social_long.dropna(subset=['Social_Value', 'ISO3', 'Year'], inplace=True)
df_social_long['Social_Value'] = pd.to_numeric(df_social_long['Social_Value'], errors='coerce')
df_social_long.dropna(subset=['Social_Value'], inplace=True)

df_social_long = df_social_long.rename(columns={'Series Name': 'Social_Series_Name'})

df_social_long['Year'] = df_social_long['Year'].astype(int)
df_merged_pop_res['Year'] = df_merged_pop_res['Year'].astype(int)

df_social_wide = df_social_long.pivot_table(
    index=['ISO3', 'Year'],
    columns='Social_Series_Name',
    values='Social_Value',
    aggfunc='mean'
).reset_index()

df_social_wide.columns.name = None
df_social_wide.columns = [str(col).replace(" ", "_").replace("/", "_per_") for col in df_social_wide.columns]

df_merged_final = pd.merge(df_merged_pop_res, df_social_wide, on=['ISO3', 'Year'], how='left')

#print(df_merged_final)
print("Shape:", df_merged_final.shape)
print(df_merged_final)


Shape: (600, 18)
    ISO3  Year  Rural_population  Total_Population_Both_sexes  \
0    AFG  2000         15657.474                    20130.327   
1    AFG  2001         16318.324                    20284.307   
2    AFG  2002         17086.910                    21378.117   
3    AFG  2003         17909.063                    22733.049   
4    AFG  2004         18692.107                    23560.654   
..   ...   ...               ...                          ...   
595  USA  2019         57727.196                   337790.067   
596  USA  2020         57456.395                   339436.159   
597  USA  2021         57168.078                   340161.441   
598  USA  2022         56862.780                   341534.045   
599  USA  2023         56539.549                   343477.335   

     Total_Population_Female  Total_Population_Male  Urban_population  \
0                  10035.682              10094.646          4436.282   
1                  10104.514              10179.792     

In [ ]:
#7

merged_all['Year'] = merged_all['Year'].astype(int)
df_merged_final['Year'] = df_merged_final['Year'].astype(int)

merged_all = pd.merge(
    merged_all,
    df_merged_final,
    on=['ISO3', 'Year'],
    how='left',
    suffixes=('_crop_disaster', '_pop_res_social')
)

print("Columns in final merged dataset:", merged_all.columns.tolist())
print("Shape:", merged_all.shape)
print(merged_all)


Columns in final merged dataset: ['ISO3', 'Year', 'Barley_Area_harvested', 'Barley_Production', 'Barley_Yield', 'Hen_eggs_in_shell,_fresh_Laying', 'Hen_eggs_in_shell,_fresh_Production', 'Hen_eggs_in_shell,_fresh_Yield', 'Hen_eggs_in_shell,_fresh_Yield/Carcass_Weight', 'Meat_of_cattle_with_the_bone,_fresh_or_chilled_Producing_Animals/Slaughtered', 'Meat_of_cattle_with_the_bone,_fresh_or_chilled_Production', 'Meat_of_cattle_with_the_bone,_fresh_or_chilled_Yield/Carcass_Weight', 'Meat_of_chickens,_fresh_or_chilled_Producing_Animals/Slaughtered', 'Meat_of_chickens,_fresh_or_chilled_Production', 'Meat_of_chickens,_fresh_or_chilled_Yield/Carcass_Weight', 'Meat_of_sheep,_fresh_or_chilled_Producing_Animals/Slaughtered', 'Meat_of_sheep,_fresh_or_chilled_Production', 'Meat_of_sheep,_fresh_or_chilled_Yield/Carcass_Weight', 'Onions_and_shallots,_dry_(excluding_dehydrated)_Area_harvested', 'Onions_and_shallots,_dry_(excluding_dehydrated)_Production', 'Onions_and_shallots,_dry_(excluding_dehydrated)

In [ ]:
df=merged_all.copy()

def clean_name(col):
    return col.replace("_crop_disaster","").replace("_pop_res_social","")

col_map = defaultdict(list)
for col in df.columns:
    base = clean_name(col)
    col_map[base].append(col)

to_keep = {}
for base, cols in col_map.items():
    if any("_crop_disaster" in c for c in cols):
        keep_col = [c for c in cols if "_crop_disaster" in c][0]
    elif any("_pop_res_social" in c for c in cols):
        keep_col = [c for c in cols if "_pop_res_social" in c][0]
    else:
        keep_col = cols[0]
    to_keep[base] = keep_col

crop_disaster_pop_res_social_merged = df[list(to_keep.values())].rename(columns={v: k for k, v in to_keep.items()})

crop_disaster_pop_res_social_merged.to_csv("crop_disaster_pop_res_social.csv", index=False, encoding="utf-8")



In [ ]:
#IMPORT 2000 - 2024

file1 = "2000-2012_Import.csv"
file2 = "2013-2024_Import.csv"

df1 = pd.read_csv(file1, encoding="latin1")
df2 = pd.read_csv(file2, encoding="latin1")

df2 = df2[df1.columns]

merged_df = pd.concat([df1, df2], ignore_index=True)

merged_df.to_csv("2000-2024_Import_merged.csv", index=False, encoding="utf-8")
print("Merged successfully without shifting columns!")


Merged successfully without shifting columns!


In [ ]:
# EXPORT 2000 - 2024

file1 = "2000-2012_Export.csv"
file2 = "2013-2024_Export.csv"

df1 = pd.read_csv(file1, encoding="latin1")
df2 = pd.read_csv(file2, encoding="latin1")

merged_df = pd.concat([df1, df2], ignore_index=True)

merged_df.to_csv("2000-2024_Export_merged.csv", index=False, encoding="utf-8")

print("CSV files merged successfully into '2000-2024_Export_merged.csv'")


CSV files merged successfully into '2000-2024_Export_merged.csv'


In [ ]:
#IMPORT + EXPORT

import_file = "2000-2024_Import_merged.csv"
export_file = "2000-2024_Export_merged.csv"

df_import = pd.read_csv(import_file)
df_export = pd.read_csv(export_file)

trade_cols = [
    'reporterISO',
    'refYear',
    'partnerISO',
    'flowDesc',
    'cifvalue',
    'fobvalue',
    'primaryValue',
]

df_import = df_import[trade_cols]
df_export = df_export[trade_cols]

trade_df = pd.concat([df_import, df_export], ignore_index=True)
trade_df = trade_df[["reporterISO","partnerISO","refYear","flowDesc","primaryValue"]].copy()

trade_df["primaryValue"] = pd.to_numeric(trade_df["primaryValue"], errors="coerce").fillna(0.0)
trade = trade_df[trade_df["primaryValue"] >= 0]

agg_keys = ["reporterISO","partnerISO","refYear","flowDesc"]
trade_year = trade.groupby(agg_keys, as_index=False)["primaryValue"].sum()

trade.to_csv("Import_export.csv", index=False, encoding="utf-8")

print("Shape:", trade_df.shape)


Shape: (177498, 5)


In [ ]:
#ALL MERGED WITHOUT INTERSECTION BEFORE PIVOT

socio_df = crop_disaster_pop_res_social_merged.copy()

trade_df = trade_df.rename(columns={"reporterISO": "ISO3", "refYear": "Year"})

trade_df["Year"] = trade_df["Year"].astype(int)
socio_df["Year"] = socio_df["Year"].astype(int)

merged_df = pd.merge(
    trade_df,
    socio_df,
    on=["ISO3", "Year"],
    how="left"
)

print("Merged shape:", merged_df.shape)
print(merged_df.head())


Merged shape: (177498, 67)
  ISO3 partnerISO  Year flowDesc  primaryValue  Barley_Area_harvested  \
0  ALB        ARG  2000   Import      162552.0                    NaN   
1  ALB        AUS  2000   Import      776949.0                    NaN   
2  ALB        BEL  2000   Import     2936268.0                    NaN   
3  ALB        CAN  2000   Import     1463840.0                    NaN   
4  ALB        CHN  2000   Import    14696573.0                    NaN   

   Barley_Production  Barley_Yield  Hen_eggs_in_shell,_fresh_Laying  \
0                NaN           NaN                              NaN   
1                NaN           NaN                              NaN   
2                NaN           NaN                              NaN   
3                NaN           NaN                              NaN   
4                NaN           NaN                              NaN   

   Hen_eggs_in_shell,_fresh_Production  ...  External_debt_stocks_(%_of_GNI)  \
0                          

In [ ]:
#ALL MERGED WITHOUT INTERSECTION AFTER PIVOT

df = merged_df.copy()

df["Year"] = df["Year"].astype(int)

pivot_trade = df.pivot_table(
    index=["ISO3", "Year", "flowDesc"],
    columns="partnerISO",
    values="primaryValue",
    aggfunc="sum"
)

pivot_trade.columns = [f"{col}_trade" for col in pivot_trade.columns]
pivot_trade = pivot_trade.reset_index()

socio_cols = df.drop(columns=["partnerISO", "primaryValue"]).drop_duplicates(
    subset=["ISO3", "Year", "flowDesc"]
)

final_df = pd.merge(pivot_trade, socio_cols, on=["ISO3", "Year", "flowDesc"], how="left")

final_df.to_csv("All.csv", index=False)

print(final_df.head())


  ISO3  Year flowDesc  AFG_trade  ARG_trade  AUS_trade  BEL_trade  BGD_trade  \
0  ABW  2000   Export        NaN    56184.0    13358.0  1271777.0        NaN   
1  ABW  2000   Import        NaN  4335972.0   566385.0  1248464.0        NaN   
2  ABW  2001   Export        NaN       45.0   185872.0        NaN        NaN   
3  ABW  2001   Import        NaN  4182681.0   546827.0  1270056.0        NaN   
4  ABW  2002   Export        NaN    42341.0   492788.0     3257.0        NaN   

   CAN_trade  CHN_trade  ...  External_debt_stocks_(%_of_GNI)  \
0    90989.0    33045.0  ...                              NaN   
1  7671112.0  4634285.0  ...                              NaN   
2    32877.0    59263.0  ...                              NaN   
3  5431899.0  5825542.0  ...                              NaN   
4    55352.0    99251.0  ...                              NaN   

   Foreign_direct_investment,_net_inflows_(%_of_GDP)  Gini_index  \
0                                                NaN        

In [ ]:
#FINAL MERGED WITH INTERSECTION BEFORE PIVOT

trade_df = pd.read_csv("Import_export.csv")
socio_df = pd.read_csv("crop_disaster_pop_res_social.csv")

trade_df = trade_df.rename(columns={"reporterISO": "ISO3"})

trade_df["refYear"] = trade_df["refYear"].astype(int)
socio_df["Year"] = socio_df["Year"].astype(int)

common_countries = set(trade_df["ISO3"]).intersection(set(socio_df["ISO3"]))

trade_df = trade_df[trade_df["ISO3"].isin(common_countries)]
socio_df = socio_df[socio_df["ISO3"].isin(common_countries)]

merged_df = pd.merge(
    trade_df,
    socio_df,
    left_on=["ISO3", "refYear"],
    right_on=["ISO3", "Year"],
    how="inner"
)

merged_df.to_csv("Trade_Socio_Intersect.csv", index=False)

print("Merged dataset saved as Trade_Socio_Intersect.csv")
print("Shape:", merged_df.shape)
print("Countries included:", sorted(common_countries))

Merged dataset saved as Trade_Socio_Intersect.csv
Shape: (24738, 68)
Countries included: ['AFG', 'ARG', 'AUS', 'BEL', 'BGD', 'CAN', 'CHN', 'DEU', 'ESP', 'FRA', 'GBR', 'HRV', 'IND', 'IRN', 'IRQ', 'ISR', 'ITA', 'JPN', 'LKA', 'PAK', 'PRT', 'RUS', 'SAU', 'SWE', 'USA']


In [ ]:
#FINAL MERGED WITH INTERSECTION AFTER PIVOT

df = pd.read_csv("Trade_Socio_Intersect.csv")

pivot_trade = df.pivot_table(
    index=["ISO3", "refYear"],
    columns="partnerISO",
    values="primaryValue",
    aggfunc="sum",
    fill_value=0
).reset_index()

pivot_trade.columns = [
    str(col) if isinstance(col, str) else f"Partner_{col}"
    for col in pivot_trade.columns
]

socio = df.drop(columns=["partnerISO", "primaryValue"]).drop_duplicates(
    subset=["ISO3", "refYear"]
)

final_df = pd.merge(
    socio,
    pivot_trade,
    on=["ISO3", "refYear"],
    how="inner"
)

final_df.to_csv("Final.csv", index=False)
print("Shape:", final_df.shape)


Shape: (527, 91)


In [23]:
#Final File

def merge_and_pivot_trade_crop_data(crop_file: str, trade_file: str, output_file: str = None) -> pd.DataFrame:
    try:
        crop_df = pd.read_csv(crop_file)
        trade_df = pd.read_csv(trade_file)

    except Exception as e:
        print(f"Error loading files: {e}")
        return None

    crop_countries = set(crop_df['ISO3'].unique())
    trade_countries = set(trade_df['reporterISO'].unique())
    common_countries = crop_countries.intersection(trade_countries)

    crop_years = set(crop_df['Year'].unique())
    trade_years = set(trade_df['refYear'].unique())
    common_years = crop_years.intersection(trade_years)

    crop_filtered = crop_df[crop_df['ISO3'].isin(common_countries)].copy()

    trade_filtered = trade_df[trade_df['reporterISO'].isin(common_countries)].copy()

    trade_filtered = trade_filtered.rename(columns={
        'reporterISO': 'ISO3',
        'refYear': 'Year'
    })

    trade_agg = trade_filtered.groupby(['ISO3', 'Year', 'partnerISO', 'flowDesc'])['primaryValue'].sum().reset_index()

    try:
        trade_agg['partner_flow'] = trade_agg['partnerISO'] + '_' + trade_agg['flowDesc']

        trade_pivot = trade_agg.pivot_table(
            index=['ISO3', 'Year'],
            columns='partner_flow',
            values='primaryValue',
            fill_value=0,
            aggfunc='sum'
        ).reset_index()

        trade_pivot.columns.name = None

        trade_cols = [col for col in trade_pivot.columns if col not in ['ISO3', 'Year']]

    except Exception as e:
        print(f"Error in pivoting: {e}")
        return None

    merged_df = crop_filtered.merge(
        trade_pivot,
        on=['ISO3', 'Year'],
        how='inner'
    )

    merged_df = merged_df.sort_values(['ISO3', 'Year']).reset_index(drop=True)

    id_cols = ['ISO3', 'Year']
    crop_socio_cols = [col for col in crop_filtered.columns if col not in id_cols]
    trade_cols = [col for col in trade_pivot.columns if col not in id_cols]

    final_column_order = id_cols + crop_socio_cols + trade_cols
    merged_df = merged_df[final_column_order]

    if output_file is None:
        output_file = "merged_crop_trade_data.csv"

    try:
        merged_df.to_csv(output_file, index=False)
    except Exception as e:
        print(f"Error saving file: {e}")

    flow_types = set()
    partner_countries = set()
    for col in trade_cols:
        if '_' in col:
            partner, flow = col.rsplit('_', 1)
            flow_types.add(flow)
            partner_countries.add(partner)

    missing_percentage = (merged_df.isnull().sum().sum() / merged_df.size) * 100

    sample_cols = ['ISO3', 'Year'] + crop_socio_cols[:3] + trade_cols[:3]

    return merged_df

def analyze_trade_patterns(merged_df: pd.DataFrame, top_n: int = 10):
    trade_cols = [col for col in merged_df.columns
                  if col not in ['ISO3', 'Year'] and
                  any(flow in col for flow in ['Import', 'Export'])]

    if not trade_cols:
        print("No trade columns found for analysis.")
        return

    import_cols = [col for col in trade_cols if 'Import' in col]
    export_cols = [col for col in trade_cols if 'Export' in col]

    if import_cols:
        total_imports = merged_df[import_cols].sum(axis=1)
        top_importers = merged_df.groupby('ISO3')['Year'].count().sort_values(ascending=False).head(top_n)
        for country, count in top_importers.items():
            print(f"  • {country}: {count} year-records")

    if trade_cols:
        trade_sums = merged_df[trade_cols].sum().sort_values(ascending=False)
        for i, (partnership, value) in enumerate(trade_sums.head(top_n).items()):
            if value > 0:
                print(f"  {i+1}. {partnership}: ${value:,.0f}")

if __name__ == "__main__":

    try:
        crop_file = "crop_disaster_pop_res_social.csv"
        trade_file = "Import_export.csv"
        merged_data = merge_and_pivot_trade_crop_data(
            crop_file=crop_file,
            trade_file=trade_file,
            output_file="final_merged_dataset.csv"
        )

        if merged_data is not None:
            analyze_trade_patterns(merged_data)

    except Exception as e:
        print(f"Error in main execution: {e}")
        import traceback
        traceback.print_exc()

  • ARG: 23 year-records
  • DEU: 23 year-records
  • AUS: 23 year-records
  • BEL: 23 year-records
  • CAN: 23 year-records
  • ESP: 23 year-records
  • CHN: 23 year-records
  • GBR: 23 year-records
  • FRA: 23 year-records
  • USA: 23 year-records
  1. USA_Export: $25,776,974,474,056
  2. CHN_Import: $24,746,202,734,708
  3. USA_Import: $15,609,990,842,800
  4. DEU_Import: $14,543,556,387,145
  5. DEU_Export: $10,297,274,931,462
  6. CHN_Export: $10,260,359,446,999
  7. JPN_Import: $9,300,171,374,712
  8. FRA_Export: $8,703,085,201,728
  9. CAN_Import: $8,522,540,470,509
  10. GBR_Export: $7,927,650,685,935


**CLEANING**

In [25]:
#CLEANED DATASET

df = pd.read_csv("final_merged_dataset.csv")

df = df.sort_values(by=["ISO3", "Year"])
df = df.groupby("ISO3").apply(lambda g: g.ffill().bfill()).reset_index(drop=True)

for col in df.select_dtypes(include=[np.number]).columns:
    df[col] = df[col].fillna(df[col].median())

for col in df.select_dtypes(include=["object"]).columns:
    if df[col].isna().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0])

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
non_numeric_cols = [c for c in df.columns if c not in numeric_cols]

df_numeric = df.groupby(["ISO3", "Year"], as_index=False)[numeric_cols].median()
df_non_numeric = df.groupby(["ISO3", "Year"], as_index=False)[non_numeric_cols].first()

df = pd.merge(df_numeric, df_non_numeric, on=["ISO3", "Year"], how="inner")

money_cols = [col for col in df.columns if any(x in col.lower()
                  for x in ["gdp", "trade", "import", "export", "value"])]

for col in money_cols:
    df[col] = df[col] / 1e6

percent_cols = [col for col in df.columns if "%" in col or "percent" in col.lower() or "unemployment" in col.lower() or "inflation" in col.lower()]

for col in percent_cols:
    df[col] = df[col].clip(lower=0, upper=100)

df["Year"] = df["Year"].astype(int)

df.to_csv("final_cleaned_dataset.csv", index=False)
print("Cleaned dataset saved as final_cleaned_dataset.csv")


Cleaned dataset saved as final_cleaned_dataset.csv
